https://towardsdatascience.com/preprocessing-text-data-using-python-576206753c28

https://towardsdatascience.com/nlp-part-3-exploratory-data-analysis-of-text-data-1caa8ab3f79d

In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import fasttext
import spacy
import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from nltk import word_tokenize, pos_tag
plt.xticks(rotation=70)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)
%matplotlib inline
import importlib
import pickle
import pyLDAvis.sklearn
from collections import Counter
from textblob import TextBlob
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)


C:\Users\mwamb\Anaconda3\lib\site-packages\past\builtins\misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping


In [2]:
def file_reader(file):
    pages = p.PdfFileReader(file)
    p_numbers = pages.getNumPages()
    text = ""
    
    for p_number in range(p_numbers):
        page = pages.getPage(p_number)
        text += page.extractText()
        
    return text

In [3]:
import PyPDF2 as p

WFP_data = file_reader("C:/Users/mwamb/Desktop/Desktop/tenthdimensionanalytics/WFP/WFP_result.pdf")


In [4]:
WFP_data

'FOOD SECURITY \n AND NUTRITION \nIN THE WORLD\nTHE STATE OF \nSAFEGUARDING AGAINST \nECONOMIC SLOWDOWNS \n AND DOWNTURNSDEMOCRATIC REPUBLIC OF THE CONGO.\n Young woman selling fruit at a street market.\nCOVER PHOTOGRAPH\n ©Shutterstock/Valeriya Anufriyeva\nThis flagship publication is part of \nTHE STATE OF THE WORLD\n series of the Food and Agriculture Organization of \nthe United Nations.\nRequired citation:\nFAO, IFAD, UNICEF, WFP and WHO. 2019. \nThe State of Food Security and Nutrition in the World 2019\n.  Safeguarding against economic slowdowns and downturns\n. Rome, FAO.\nLicence: CC BY-NC-SA 3.0 IGO.\nThe designations employed and the presentation of material in this information product do not imply the expression of any \nopinion whatsoever on the part of the Food and Agriculture Organization of the United Nations (FAO), the International \nFund for Agricultural Development (IFAD), the United Nations Children™s Fund (UNICEF), the World Food Programme \n(WFP) or the World Hea

In [5]:
clusterwfp = 'WFP'
wordswfp = WFP_data

In [6]:
df = pd.DataFrame({'cluster': clusterwfp, 'words': WFP_data}, index=[0])

In [7]:
df

,cluster,words
0,WFP,FOOD SECURITY \n AND NUTRITION \nIN THE WORLD\nTHE STATE OF \nSAFEGUARDING AGAINST \nECONOMIC SL...


In [8]:
#df.to_pickle('C:/Users/jeanl/Desktop/tenthdimensionanalytics/WFP/wfpdata')

df.to_pickle("C:/Users/mwamb/Desktop/Desktop/tenthdimensionanalytics/WFP/WFP_data")

## Text Pre-Processing

### Expanding Contractions

In [9]:
df['words_description'] = df['words'].apply(lambda x: [contractions.fix(word) for word in x.split()])
df.head()

,cluster,words,words_description
0,WFP,FOOD SECURITY \n AND NUTRITION \nIN THE WORLD\nTHE STATE OF \nSAFEGUARDING AGAINST \nECONOMIC SL...,"[FOOD, SECURITY, AND, NUTRITION, IN, THE, WORLD, THE, STATE, OF, SAFEGUARDING, AGAINST, ECONOMIC..."


In [10]:
df['words_description_str'] = [' '.join(map(str, l)) for l in df['words_description']]
df.head()

,cluster,words,words_description,words_description_str
0,WFP,FOOD SECURITY \n AND NUTRITION \nIN THE WORLD\nTHE STATE OF \nSAFEGUARDING AGAINST \nECONOMIC SL...,"[FOOD, SECURITY, AND, NUTRITION, IN, THE, WORLD, THE, STATE, OF, SAFEGUARDING, AGAINST, ECONOMIC...",FOOD SECURITY AND NUTRITION IN THE WORLD THE STATE OF SAFEGUARDING AGAINST ECONOMIC SLOWDOWNS AN...


### English Language Detection

In [11]:
pretrained_model = "C:/Users/mwamb/Downloads/lid.176.bin" 
model = fasttext.load_model(pretrained_model)
langs = []
for sent in df['words_description_str']:
    lang = model.predict(sent)[0]
    langs.append(str(lang)[11:13])
df['langs'] = langs

In [12]:
df

,cluster,words,words_description,words_description_str,langs
0,WFP,FOOD SECURITY \n AND NUTRITION \nIN THE WORLD\nTHE STATE OF \nSAFEGUARDING AGAINST \nECONOMIC SL...,"[FOOD, SECURITY, AND, NUTRITION, IN, THE, WORLD, THE, STATE, OF, SAFEGUARDING, AGAINST, ECONOMIC...",FOOD SECURITY AND NUTRITION IN THE WORLD THE STATE OF SAFEGUARDING AGAINST ECONOMIC SLOWDOWNS AN...,en


## Tokenization

In [13]:
df['tokenized'] = df['words_description_str'].apply(word_tokenize)
df.head()

,cluster,words,words_description,words_description_str,langs,tokenized
0,WFP,FOOD SECURITY \n AND NUTRITION \nIN THE WORLD\nTHE STATE OF \nSAFEGUARDING AGAINST \nECONOMIC SL...,"[FOOD, SECURITY, AND, NUTRITION, IN, THE, WORLD, THE, STATE, OF, SAFEGUARDING, AGAINST, ECONOMIC...",FOOD SECURITY AND NUTRITION IN THE WORLD THE STATE OF SAFEGUARDING AGAINST ECONOMIC SLOWDOWNS AN...,en,"[FOOD, SECURITY, AND, NUTRITION, IN, THE, WORLD, THE, STATE, OF, SAFEGUARDING, AGAINST, ECONOMIC..."


## Converting all Characters to Lowercase

In [14]:
df['lower'] = df['tokenized'].apply(lambda x: [word.lower() for word in x])
df.head()

,cluster,words,words_description,words_description_str,langs,tokenized,lower
0,WFP,FOOD SECURITY \n AND NUTRITION \nIN THE WORLD\nTHE STATE OF \nSAFEGUARDING AGAINST \nECONOMIC SL...,"[FOOD, SECURITY, AND, NUTRITION, IN, THE, WORLD, THE, STATE, OF, SAFEGUARDING, AGAINST, ECONOMIC...",FOOD SECURITY AND NUTRITION IN THE WORLD THE STATE OF SAFEGUARDING AGAINST ECONOMIC SLOWDOWNS AN...,en,"[FOOD, SECURITY, AND, NUTRITION, IN, THE, WORLD, THE, STATE, OF, SAFEGUARDING, AGAINST, ECONOMIC...","[food, security, and, nutrition, in, the, world, the, state, of, safeguarding, against, economic..."


### Removing Punctuations

In [15]:
punc = string.punctuation
df['no_punc'] = df['lower'].apply(lambda x: [word for word in x if word not in punc])
df.head()

,cluster,words,words_description,words_description_str,langs,tokenized,lower,no_punc
0,WFP,FOOD SECURITY \n AND NUTRITION \nIN THE WORLD\nTHE STATE OF \nSAFEGUARDING AGAINST \nECONOMIC SL...,"[FOOD, SECURITY, AND, NUTRITION, IN, THE, WORLD, THE, STATE, OF, SAFEGUARDING, AGAINST, ECONOMIC...",FOOD SECURITY AND NUTRITION IN THE WORLD THE STATE OF SAFEGUARDING AGAINST ECONOMIC SLOWDOWNS AN...,en,"[FOOD, SECURITY, AND, NUTRITION, IN, THE, WORLD, THE, STATE, OF, SAFEGUARDING, AGAINST, ECONOMIC...","[food, security, and, nutrition, in, the, world, the, state, of, safeguarding, against, economic...","[food, security, and, nutrition, in, the, world, the, state, of, safeguarding, against, economic..."


### Removing Stopwords

In [16]:
stop_words = set(stopwords.words('english'))
df['stopwords_removed'] = df['no_punc'].apply(lambda x: [word for word in x if word not in stop_words])
df.head()

,cluster,words,words_description,words_description_str,langs,tokenized,lower,no_punc,stopwords_removed
0,WFP,FOOD SECURITY \n AND NUTRITION \nIN THE WORLD\nTHE STATE OF \nSAFEGUARDING AGAINST \nECONOMIC SL...,"[FOOD, SECURITY, AND, NUTRITION, IN, THE, WORLD, THE, STATE, OF, SAFEGUARDING, AGAINST, ECONOMIC...",FOOD SECURITY AND NUTRITION IN THE WORLD THE STATE OF SAFEGUARDING AGAINST ECONOMIC SLOWDOWNS AN...,en,"[FOOD, SECURITY, AND, NUTRITION, IN, THE, WORLD, THE, STATE, OF, SAFEGUARDING, AGAINST, ECONOMIC...","[food, security, and, nutrition, in, the, world, the, state, of, safeguarding, against, economic...","[food, security, and, nutrition, in, the, world, the, state, of, safeguarding, against, economic...","[food, security, nutrition, world, state, safeguarding, economic, slowdowns, downturnsdemocratic..."


## Stemming vs Lemmatization

In [17]:
df['pos_tags'] = df['stopwords_removed'].apply(nltk.tag.pos_tag)
df.head()

,cluster,words,words_description,words_description_str,langs,tokenized,lower,no_punc,stopwords_removed,pos_tags
0,WFP,FOOD SECURITY \n AND NUTRITION \nIN THE WORLD\nTHE STATE OF \nSAFEGUARDING AGAINST \nECONOMIC SL...,"[FOOD, SECURITY, AND, NUTRITION, IN, THE, WORLD, THE, STATE, OF, SAFEGUARDING, AGAINST, ECONOMIC...",FOOD SECURITY AND NUTRITION IN THE WORLD THE STATE OF SAFEGUARDING AGAINST ECONOMIC SLOWDOWNS AN...,en,"[FOOD, SECURITY, AND, NUTRITION, IN, THE, WORLD, THE, STATE, OF, SAFEGUARDING, AGAINST, ECONOMIC...","[food, security, and, nutrition, in, the, world, the, state, of, safeguarding, against, economic...","[food, security, and, nutrition, in, the, world, the, state, of, safeguarding, against, economic...","[food, security, nutrition, world, state, safeguarding, economic, slowdowns, downturnsdemocratic...","[(food, NN), (security, NN), (nutrition, NN), (world, NN), (state, NN), (safeguarding, VBG), (ec..."


In [18]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
df['wordnet_pos'] = df['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
df.head()

,cluster,words,words_description,words_description_str,langs,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos
0,WFP,FOOD SECURITY \n AND NUTRITION \nIN THE WORLD\nTHE STATE OF \nSAFEGUARDING AGAINST \nECONOMIC SL...,"[FOOD, SECURITY, AND, NUTRITION, IN, THE, WORLD, THE, STATE, OF, SAFEGUARDING, AGAINST, ECONOMIC...",FOOD SECURITY AND NUTRITION IN THE WORLD THE STATE OF SAFEGUARDING AGAINST ECONOMIC SLOWDOWNS AN...,en,"[FOOD, SECURITY, AND, NUTRITION, IN, THE, WORLD, THE, STATE, OF, SAFEGUARDING, AGAINST, ECONOMIC...","[food, security, and, nutrition, in, the, world, the, state, of, safeguarding, against, economic...","[food, security, and, nutrition, in, the, world, the, state, of, safeguarding, against, economic...","[food, security, nutrition, world, state, safeguarding, economic, slowdowns, downturnsdemocratic...","[(food, NN), (security, NN), (nutrition, NN), (world, NN), (state, NN), (safeguarding, VBG), (ec...","[(food, n), (security, n), (nutrition, n), (world, n), (state, n), (safeguarding, v), (economic,..."


In [19]:
wnl = WordNetLemmatizer()
df['lemmatized'] = df['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
df.head()

,cluster,words,words_description,words_description_str,langs,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos,lemmatized
0,WFP,FOOD SECURITY \n AND NUTRITION \nIN THE WORLD\nTHE STATE OF \nSAFEGUARDING AGAINST \nECONOMIC SL...,"[FOOD, SECURITY, AND, NUTRITION, IN, THE, WORLD, THE, STATE, OF, SAFEGUARDING, AGAINST, ECONOMIC...",FOOD SECURITY AND NUTRITION IN THE WORLD THE STATE OF SAFEGUARDING AGAINST ECONOMIC SLOWDOWNS AN...,en,"[FOOD, SECURITY, AND, NUTRITION, IN, THE, WORLD, THE, STATE, OF, SAFEGUARDING, AGAINST, ECONOMIC...","[food, security, and, nutrition, in, the, world, the, state, of, safeguarding, against, economic...","[food, security, and, nutrition, in, the, world, the, state, of, safeguarding, against, economic...","[food, security, nutrition, world, state, safeguarding, economic, slowdowns, downturnsdemocratic...","[(food, NN), (security, NN), (nutrition, NN), (world, NN), (state, NN), (safeguarding, VBG), (ec...","[(food, n), (security, n), (nutrition, n), (world, n), (state, n), (safeguarding, v), (economic,...","[food, security, nutrition, world, state, safeguard, economic, slowdown, downturnsdemocratic, re..."


In [22]:
df.to_csv('WFP_clean_data.csv', index=False,header=True)


In [ ]:
#with open('indeed_scrape_clean.pkl', 'rb') as pickle_file:
#    df = pickle.load(pickle_file)

In [24]:
df.to_pickle("WFP_clean_data.pkl")

In [25]:
unpickled_df = pd.read_pickle("WFP_clean_data.pkl")
unpickled_df

,cluster,words,words_description,words_description_str,langs,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos,lemmatized
0,WFP,FOOD SECURITY \n AND NUTRITION \nIN THE WORLD\nTHE STATE OF \nSAFEGUARDING AGAINST \nECONOMIC SL...,"[FOOD, SECURITY, AND, NUTRITION, IN, THE, WORLD, THE, STATE, OF, SAFEGUARDING, AGAINST, ECONOMIC...",FOOD SECURITY AND NUTRITION IN THE WORLD THE STATE OF SAFEGUARDING AGAINST ECONOMIC SLOWDOWNS AN...,en,"[FOOD, SECURITY, AND, NUTRITION, IN, THE, WORLD, THE, STATE, OF, SAFEGUARDING, AGAINST, ECONOMIC...","[food, security, and, nutrition, in, the, world, the, state, of, safeguarding, against, economic...","[food, security, and, nutrition, in, the, world, the, state, of, safeguarding, against, economic...","[food, security, nutrition, world, state, safeguarding, economic, slowdowns, downturnsdemocratic...","[(food, NN), (security, NN), (nutrition, NN), (world, NN), (state, NN), (safeguarding, VBG), (ec...","[(food, n), (security, n), (nutrition, n), (world, n), (state, n), (safeguarding, v), (economic,...","[food, security, nutrition, world, state, safeguard, economic, slowdown, downturnsdemocratic, re..."


In [ ]:
df = df[['cluster', 'lemmatized']]
df.head()

## Sentiment Analysis

In [ ]:
df['lemma_str'] = [' '.join(map(str,l)) for l in df['lemmatized']]
df.head()

In [ ]:
df['sentiment'] = df['lemma_str'].apply(lambda x: TextBlob(x).sentiment.polarity)
df.head()

In [ ]:
df['sentiment'] = df['lemma_str'].apply(lambda x: TextBlob(x).sentiment.polarity)
df.head()

In [ ]:
plt.figure(figsize=(50,30))
plt.margins(0.02)
plt.xlabel('Sentiment', fontsize=50)
plt.xticks(fontsize=40)
plt.ylabel('Frequency', fontsize=50)
plt.yticks(fontsize=40)
plt.hist(df['sentiment'], bins=50)
plt.title('Sentiment Distribution', fontsize=60)
plt.show()

In [ ]:
x_cluster = df.cluster.value_counts()
y_cluster = x_cluster.sort_index()
plt.figure(figsize=(50,30))
sns.barplot(x_cluster.index, x_cluster.values, alpha=0.8)
plt.title("Cluster Distribution", fontsize=50)
plt.ylabel('Frequency', fontsize=50)
plt.yticks(fontsize=40)
plt.xlabel('Cluster', fontsize=50)
plt.xticks(fontsize=40)

In [ ]:
plt.figure(figsize=(30,10))
plt.title('Percentage of Cluster', fontsize=20)
df.cluster.value_counts().plot(kind='pie',
                              wedgeprops=dict(width=.7), autopct='%1.0f%%', startangle= -20, 
                              textprops={'fontsize': 15})

In [ ]:
polarity_avg = df.groupby('cluster')['sentiment'].mean().plot(kind='bar', figsize=(50,30))
plt.xlabel('Cluster', fontsize=45)
plt.ylabel('Average Sentiment', fontsize=45)
plt.xticks(fontsize=40)
plt.yticks(fontsize=40)
plt.title('Average Sentiment per Cluster Distribution', fontsize=50)
plt.show()

In [ ]:
df['word_count'] = df['lemmatized'].apply(lambda x: len(str(x).split()))
df['word_len'] = df['lemma_str'].astype(str).apply(len)

In [ ]:
df

In [ ]:
letter_avg = df.groupby('cluster')['word_len'].mean().plot(kind='bar', figsize=(50,30))
plt.xlabel('Rating', fontsize=35)
plt.ylabel('Count of Letters in cluster', fontsize=35)
plt.xticks(fontsize=40)
plt.yticks(fontsize=40)
plt.title('Average Number of Letters per Rating Distribution', fontsize=40)
plt.show()

In [ ]:
word_avg = df.groupby('cluster')['word_count'].mean().plot(kind='bar', figsize=(50,30))
plt.xlabel('Clusters', fontsize=35)
plt.ylabel('Count of Words in Rating', fontsize=35)
plt.xticks(fontsize=40)
plt.yticks(fontsize=40)
plt.title('Average Number of Words per clusters Distribution', fontsize=40)
plt.show()

## Term Frequency Analysis

In [ ]:
words = df['lemmatized']
allwords = []
for wordlist in words:
    allwords += wordlist
print(allwords)

In [ ]:
mostcommon = FreqDist(allwords).most_common(100)
wordcloud = WordCloud(width=1600, height=800, background_color='white').generate(str(mostcommon))
fig = plt.figure(figsize=(30,10), facecolor='white')
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title('Top 100 Most Common Words', fontsize=100)
plt.tight_layout(pad=0)
plt.show()

In [ ]:
mostcommon_small = FreqDist(allwords).most_common(25)
x, y = zip(*mostcommon_small)
plt.figure(figsize=(50,30))
plt.margins(0.02)
plt.bar(x, y)
plt.xlabel('Words', fontsize=50)
plt.ylabel('Frequency of Words', fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(rotation=60, fontsize=40)
plt.title('Frequency of 25 Most Common Words', fontsize=60)
plt.show()

In [ ]:
group_by = df.groupby('cluster')['lemma_str'].apply(lambda x: Counter(' '.join(x).split()).most_common(25))
group_by_0 = group_by.iloc[0]
words0 = list(zip(*group_by_0))[0]
freq0 = list(zip(*group_by_0))[1]
plt.figure(figsize=(50,30))
plt.bar(words0, freq0)
plt.xlabel('Words', fontsize=50)
plt.ylabel('Frequency of Words', fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(rotation=60, fontsize=40)
plt.title('Frequency of 25 Most Common Words for Cluster=FAO', fontsize=60)
plt.show()

## Topic Modeling

In [ ]:
#tf_vectorizer = CountVectorizer(max_df=0.9, min_df=1, max_features=0)
tf_vectorizer = CountVectorizer(max_df=1.0, min_df=1, max_features=None)
tf = tf_vectorizer.fit_transform(df['lemma_str'].values.astype('U'))
tf_feature_names = tf_vectorizer.get_feature_names()
doc_term_matrix = pd.DataFrame(tf.toarray(), columns=list(tf_feature_names))
doc_term_matrix

## Dirichlet Allocation (LDA) Topic Modeling

In [ ]:
lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', max_iter=500, random_state=0).fit(tf)
no_top_words = 10
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                          for i in topic.argsort()[:-no_top_words - 1:-1]]))
              
display_topics(lda_model, tf_feature_names, no_top_words)

## pyLDAvis

In [ ]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, tf, tf_vectorizer, mds='tsne')
panel

## TF-IDF

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, max_features=None, use_idf=True)
tfidf = tfidf_vectorizer.fit_transform(df['lemma_str'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
doc_term_matrix_tfidf = pd.DataFrame(tfidf.toarray(), columns=list(tfidf_feature_names))
doc_term_matrix_tfidf

## Non-Negative Matrix Factorization (NMF)

In [ ]:
nmf = NMF(n_components=1, random_state=0, alpha=.1, init='nndsvd').fit(tfidf)
display_topics(nmf, tfidf_feature_names, no_top_words)